In [221]:
import requests
import json
import time
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

API_KEY = '1a9ea75b5f92dfda381d817fc00e5458' 
USER_AGENT = '808e0a8bbaf9d30bf181edb60c742824'  
LIMIT = 20  ##Número de resultados a obtener

headers = {
    'user-agent': USER_AGENT
}

##sacar top artists
def get_top_artists():
    params = {
        'method': 'chart.getTopArtists',
        'api_key': API_KEY,
        'format': 'json',
        'limit': LIMIT
    }
    response = requests.get('https://ws.audioscrobbler.com/2.0/', headers=headers, params=params)
    if response.status_code == 200:
        data = response.json()
        df = pd.json_normalize(data['artists']['artist'])
    else:
        print(f"Error en la solicitud de top artistas globales: {response.status_code}")
    time.sleep(1)
    return df

##sacar top artistas por pais
def get_top_artists_by_country(country):
    params = {
        'method': 'geo.getTopArtists',
        'country': country,
        'api_key': API_KEY,
        'format': 'json',
        'limit': LIMIT
    }
    response = requests.get('https://ws.audioscrobbler.com/2.0/', headers=headers, params=params)
    if response.status_code == 200:
        data = response.json()
        df = pd.json_normalize(data['topartists']['artist'])
        country_artists = [
            {
                "name": ["name"],
                "listeners": float(name["listeners"]),
                "playcount": float(name["playcount"]),
                "country": country
            }
            for name in df
        ]
        return pd.DataFrame(country_artists)
    else:
        print(f"Error en la solicitud de top artistas por país: {response.status_code}")
    time.sleep(1)
    return df

##sacar top canciones por pais
def get_top_tracks_by_country(country):
    params = {
        'method': 'geo.getTopTracks',
        'country': country,
        'api_key': API_KEY,
        'format': 'json',
        'limit': LIMIT
    }
    response = requests.get('https://ws.audioscrobbler.com/2.0/', headers=headers, params=params)
    if response.status_code == 200:
        data = response.json()
        df = pd.json_normalize(data['tracks']['track'])
    else:
        print(f"Error en la solicitud de top canciones por país: {response.status_code}")
    time.sleep(1)
    return df

##sacar top album de x artista
def get_top_albums_by_artist(artist):
    params = {
        'method': 'artist.getTopAlbums',
        'artist': artist,
        'api_key': API_KEY,
        'format': 'json',
        'limit': LIMIT
    }
    response = requests.get('https://ws.audioscrobbler.com/2.0/', headers=headers, params=params)
    if response.status_code == 200:
        data = response.json()
        df = pd.json_normalize(data['topalbums']['album'])
    else:
        print(f"Error en la solicitud de top álbumes del artista: {response.status_code}")
    time.sleep(1)
    return df

##limpia y ordena dataframe
def limpiar_y_ordenar(df, columnas_a_eliminar=None):
    if columnas_a_eliminar:
        df = df.drop(columns=columnas_a_eliminar)
    if 'listeners' in df.columns:
        df = df.sort_values(by='listeners', ascending=False)
    df = df.reset_index(drop=True)
    return df

In [ ]:
df_top_artist = get_top_artists()
df_top_artist

In [ ]:
limpiar_y_ordenar(df_top_artist, ["mbid", "url", "streamable", "image"])

In [ ]:
# Crear gráfica de barras agrupadas
plt.figure(figsize=(10, 6))
sns.barplot(x="name", y="listeners", data=df_top_artist, color="green", label="Oyentes")
sns.barplot(x="name", y="playcount", data=df_top_artist, color="skyblue", alpha=0.7, label="Reproducciones")

# Personalización del gráfico
plt.xlabel("Artistas")
plt.ylabel("Cantidad")
plt.title("Oyentes y Reproducciones por Artista")
plt.legend()
plt.xticks(rotation=45)  # Rotar los nombres para mejor legibilidad
plt.show()

In [ ]:
# plt.figure(figsize=(10, 6))
# sns.barplot(x="listeners", y="playcount", hue="name", data=df_top_artist, s=100)

# plt.xlabel("Oyentes")
# plt.ylabel("Reproducciones")
# plt.title("Relación entre Oyentes y Reproducciones por Artista")
# plt.show()

fig, ax = plt.subplots(figsize = (10, 6))

sns.barplot(x = "listeners", y = "playcount", hue = "name", data = df_top_artist, palette = "rainbow")

ax.legend_.remove()
plt.show()

In [ ]:
# Crear gráfico de líneas
plt.figure(figsize=(10, 6))
sns.lineplot(x="name", y="playcount", data=df_top_artist, marker="o", label="Reproducciones", color="blue")
sns.lineplot(x="name", y="listeners", data=df_top_artist, marker="s", label="Oyentes", color="red")

# Personalización del gráfico
plt.xlabel("Fecha")
plt.ylabel("Cantidad")
plt.title("Relación entre Reproducciones y Oyentes")
plt.legend()
plt.xticks(rotation=45)
plt.grid()

plt.show()

In [223]:
get_top_artists_by_country("Spain")

TypeError: string indices must be integers, not 'str'

In [189]:
df_artist_DE = get_top_artists_by_country("Germany")
df_artist_ES = get_top_artists_by_country("Spain")
df_artist_FR = get_top_artists_by_country("France")
df_artist_RUS = get_top_artists_by_country("Russian Federation")
df_artist_UK = get_top_artists_by_country("United Kingdom")
df_artist_USA = get_top_artists_by_country("United States")

df_artist_DE_clean = limpiar_y_ordenar(df_artist_DE, ["mbid", "url", "streamable", "image"])
df_artist_ES_clean = limpiar_y_ordenar(df_artist_ES, ["mbid", "url", "streamable", "image"])
df_artist_FR_clean = limpiar_y_ordenar(df_artist_FR, ["mbid", "url", "streamable", "image"])
df_artist_RUS_clean = limpiar_y_ordenar(df_artist_RUS, ["mbid", "url", "streamable", "image"])
df_artist_UK_clean = limpiar_y_ordenar(df_artist_UK, ["mbid", "url", "streamable", "image"])
df_artist_USA_clean = limpiar_y_ordenar(df_artist_USA, ["mbid", "url", "streamable", "image"])

df_artist_DE_clean["Country"] = "Germany"
df_artist_ES_clean["Country"] = "Spain"
df_artist_FR_clean["Country"] = "France"
df_artist_RUS_clean["Country"] = "Russia"
df_artist_UK_clean["Country"] = "UK"
df_artist_USA_clean["Country"] = "USA"

df_artist_DE_clean["listeners"] = df_artist_DE_clean["listeners"].astype(float)
df_artist_ES_clean["listeners"] = df_artist_ES_clean["listeners"].astype(float)
df_artist_FR_clean["listeners"] = df_artist_FR_clean["listeners"].astype(float)
df_artist_RUS_clean["listeners"] = df_artist_RUS_clean["listeners"].astype(float)
df_artist_UK_clean["listeners"] = df_artist_UK_clean["listeners"].astype(float)
df_artist_USA_clean["listeners"] = df_artist_USA_clean["listeners"].astype(float)

In [160]:
# df_artist_DE_clean.set_index("Country", inplace=True)
# df_artist_DE_clean.reset_index(inplace=True)

In [190]:
df_artist_DE_clean

,name,listeners,Country
0,Coldplay,8023436.0,Germany
1,Radiohead,7169431.0,Germany
2,Eminem,7015942.0,Germany
3,Nirvana,6589164.0,Germany
4,Red Hot Chili Peppers,6430577.0,Germany
5,Queen,6353361.0,Germany
6,Linkin Park,5968935.0,Germany
7,Michael Jackson,5849794.0,Germany
8,The Beatles,5749528.0,Germany
9,Foo Fighters,5672554.0,Germany


In [193]:
df_artist_ES_clean

,name,listeners,Country
0,Coldplay,8023436.0,Spain
1,Radiohead,7169431.0,Spain
2,Nirvana,6589164.0,Spain
3,Red Hot Chili Peppers,6430577.0,Spain
4,Queen,6353361.0,Spain
5,Arctic Monkeys,6244597.0,Spain
6,Michael Jackson,5849794.0,Spain
7,The Beatles,5749528.0,Spain
8,Daft Punk,5590956.0,Spain
9,Muse,5570366.0,Spain


In [195]:
df_artist_FR_clean

,name,listeners,Country
0,Coldplay,8023436.0,France
1,Rihanna,7326583.0,France
2,Radiohead,7169431.0,France
3,Nirvana,6589164.0,France
4,Red Hot Chili Peppers,6430577.0,France
5,Queen,6353361.0,France
6,Gorillaz,5937746.0,France
7,Michael Jackson,5849794.0,France
8,The Beatles,5749528.0,France
9,Daft Punk,5590956.0,France


In [197]:
df_artist_RUS_clean

,name,listeners,Country
0,Coldplay,8023436.0,Russia
1,Radiohead,7171541.0,Russia
2,Nirvana,6589164.0,Russia
3,Red Hot Chili Peppers,6430577.0,Russia
4,Queen,6353361.0,Russia
5,The Killers,6158018.0,Russia
6,Linkin Park,5968935.0,Russia
7,Gorillaz,5936188.0,Russia
8,The Beatles,5750828.0,Russia
9,Daft Punk,5590956.0,Russia


In [199]:
df_artist_UK_clean

,name,listeners,Country
0,Coldplay,8023436.0,UK
1,Radiohead,7169431.0,UK
2,Nirvana,6590719.0,UK
3,Red Hot Chili Peppers,6428887.0,UK
4,Queen,6353361.0,UK
5,Arctic Monkeys,6244597.0,UK
6,The Killers,6158018.0,UK
7,Michael Jackson,5849794.0,UK
8,The Beatles,5749528.0,UK
9,Foo Fighters,5672554.0,UK


In [201]:
df_artist_USA_clean

,name,listeners,Country
0,Coldplay,8023436.0,USA
1,Rihanna,7326583.0,USA
2,Radiohead,7169431.0,USA
3,Kanye West,7143728.0,USA
4,Nirvana,6589164.0,USA
5,Red Hot Chili Peppers,6430577.0,USA
6,Queen,6353361.0,USA
7,Drake,6042846.0,USA
8,Gorillaz,5937746.0,USA
9,Michael Jackson,5849794.0,USA


In [ ]:
artistas = df_artist_DE_clean['name'].unique()
 
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(18, 10))

axes = axes.flatten()
 
for i, artista in enumerate(artistas):

    datos_artista = df_top_canciones[df_top_canciones['artist.name'] == artista]

    sns.barplot(x='playcount', y='name', data=datos_artista, ax=axes[i])

    axes[i].set_title(f'Reproducciones de {artista}')

    axes[i].set_xlabel('Reproducciones')

    axes[i].set_ylabel('Canción')
 
plt.tight_layout()

plt.show()
 

In [125]:
df_sia = get_top_albums_by_artist("Sia")
df_billie = get_top_albums_by_artist("Billie Eilish")